In [76]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st
import csv
import json
import gmaps
#importing libraries

In [77]:
#Checking crime categories
url_crime_cat = "https://data.police.uk/api/crime-categories?date=2020"
response_crime_cat = requests.get(url_crime_cat).json()
response_crime_cat

[{'url': 'all-crime', 'name': 'All crime'},
 {'url': 'anti-social-behaviour', 'name': 'Anti-social behaviour'},
 {'url': 'bicycle-theft', 'name': 'Bicycle theft'},
 {'url': 'burglary', 'name': 'Burglary'},
 {'url': 'criminal-damage-arson', 'name': 'Criminal damage and arson'},
 {'url': 'drugs', 'name': 'Drugs'},
 {'url': 'other-theft', 'name': 'Other theft'},
 {'url': 'possession-of-weapons', 'name': 'Possession of weapons'},
 {'url': 'public-order', 'name': 'Public order'},
 {'url': 'robbery', 'name': 'Robbery'},
 {'url': 'shoplifting', 'name': 'Shoplifting'},
 {'url': 'theft-from-the-person', 'name': 'Theft from the person'},
 {'url': 'vehicle-crime', 'name': 'Vehicle crime'},
 {'url': 'violent-crime', 'name': 'Violence and sexual offences'},
 {'url': 'other-crime', 'name': 'Other crime'}]

In [78]:
#checking Police Forces that exist
url_forces = "https://data.police.uk/api/forces"
response_forces = requests.get(url_forces).json()
response_forces


[{'id': 'avon-and-somerset', 'name': 'Avon and Somerset Constabulary'},
 {'id': 'bedfordshire', 'name': 'Bedfordshire Police'},
 {'id': 'cambridgeshire', 'name': 'Cambridgeshire Constabulary'},
 {'id': 'cheshire', 'name': 'Cheshire Constabulary'},
 {'id': 'city-of-london', 'name': 'City of London Police'},
 {'id': 'cleveland', 'name': 'Cleveland Police'},
 {'id': 'cumbria', 'name': 'Cumbria Constabulary'},
 {'id': 'derbyshire', 'name': 'Derbyshire Constabulary'},
 {'id': 'devon-and-cornwall', 'name': 'Devon & Cornwall Police'},
 {'id': 'dorset', 'name': 'Dorset Police'},
 {'id': 'durham', 'name': 'Durham Constabulary'},
 {'id': 'dyfed-powys', 'name': 'Dyfed-Powys Police'},
 {'id': 'essex', 'name': 'Essex Police'},
 {'id': 'gloucestershire', 'name': 'Gloucestershire Constabulary'},
 {'id': 'greater-manchester', 'name': 'Greater Manchester Police'},
 {'id': 'gwent', 'name': 'Gwent Police'},
 {'id': 'hampshire', 'name': 'Hampshire Constabulary'},
 {'id': 'hertfordshire', 'name': 'Hertford

In [56]:
#Creating a function for concatetning a URL into a format that can be parsed by API call 
#it takes 3 variables: URL string, date,and shortened list of coordinates 
#cuz it crashed if there are more than ~ 140 coordinates in an URl
def fun_gigastr(gigastr,date,lat_lon_zipped_shortened):
    try:
        for i in range(len(lat_lon_zipped_shortened)):
            for j in range(2):
                if j == 0:
                    a = ","
                    gigastr = gigastr  + str(lat_lon_zipped_shortened[i][j]) + a
                else:
                    a = ":"
                    gigastr = gigastr  + str(lat_lon_zipped_shortened[i][j]) + a
    finally:
        gigastr = gigastr + date
    return gigastr

In [71]:
#Our Parser, if you want to retrieve a data from different API call like stop and search or else, just change the link.
#Feel free to add any other lists or whatever you need to retrieve the data that you need
data ={}
data_category =[]
data_lat = []
data_lon = []
data_outcome = []
data_month = []




gigastr ="https://data.police.uk/api/crimes-street/all-crime?poly="
date_number = ["01","02","03","04","05","06","07","08","09","10","11","12"]

west_midlands_neighbourhoods = []
west_midlands_ids = []

#URL for the list of existing neighbourhoods in "west-midlands"

url_west_midlands_police_neighbourhoods = "https://data.police.uk/api/west-midlands/neighbourhoods"
response_west_midlands_police_neighbourhoods = requests.get(url_west_midlands_police_neighbourhoods).json()

#Retriving neighbourhoods
for u in response_west_midlands_police_neighbourhoods:
    west_midlands_neighbourhoods.append(u)
    
#Getting Neighbourhoods IDs that we will insert into function
for k in range(len(west_midlands_neighbourhoods)):
    west_midlands_ids.append(west_midlands_neighbourhoods[k]["id"])

#The Parser begins here: first we get boundaries coordinates for each boundary from API   
for h in west_midlands_ids:

    url_boundary = f"https://data.police.uk/api/west-midlands/{h}/boundary"
    response_boundary = requests.get(url_boundary).json()
    print(f"---Trying to get data for : {h}---")
    
    lat_lon_pairs = []
    lat = []
    lon = []

#We insert lat and lon in separate lists
    for o in response_boundary:
        lat_lon_pairs.append(o)
#We zip them together in pairs to form a coresponding pair
    for p in range(len(lat_lon_pairs)):  
        lat.append(lat_lon_pairs[p]["latitude"])
        lon.append(lat_lon_pairs[p]["longitude"])
        lat_lon_zipped = list(zip(lat,lon))
#Now we have list of pairs and individual values for ease of use. Tiny chance it might be redundant a bit but whatever lol

#Now we shorten the coordinate list cuz after ~140 pair in an url it will crash.
    for j in date_number:
        if len(lat_lon_zipped)>4500:
            lat_lon_zipped_shortened = lat_lon_zipped[::100]
        if len(lat_lon_zipped)>4000:
            lat_lon_zipped_shortened = lat_lon_zipped[::40]
        if len(lat_lon_zipped)>3500:
            lat_lon_zipped_shortened = lat_lon_zipped[::35]
        if len(lat_lon_zipped)>3000:
            lat_lon_zipped_shortened = lat_lon_zipped[::30]
        if len(lat_lon_zipped)>2500:
            lat_lon_zipped_shortened = lat_lon_zipped[::25]    
        if len(lat_lon_zipped)>2000:
            lat_lon_zipped_shortened = lat_lon_zipped[::20]    
        if len(lat_lon_zipped)>1500:
            lat_lon_zipped_shortened = lat_lon_zipped[::15]
        else:
            lat_lon_zipped_shortened = lat_lon_zipped[::10]
        print(f"We've got {len(lat_lon_zipped)} coordinates to work with")    
        print(f"Total coordinates that we work with is: {len(lat_lon_zipped_shortened)}")
#Now we start looping over the date and our function, getting data. Feel free to change the url in a function or make a new one
#Do note it might easily take you ~10-15 minutes for the function to finish. Go brew yourself a coffee or smthing.
        date = f"&date=2020-{j}"
        print(f"Date is: {date}")
        fun_gigastr(gigastr,date,lat_lon_zipped_shortened)
        response_crime = requests.get(fun_gigastr(gigastr,date,lat_lon_zipped_shortened)).json()   
#This is a tricky one, now we append our lists of data with Try function and if there is no data point for it
#then it will use Except to add "NaN". 
#Neighbourhood will be in only the first row, feel free to change this chunk to add it in all rows if you find a way how to.  


        for i in range(len(response_crime)):
            try:
                data_category.append(response_crime[i]["category"])
                data["Category"] = data_category
            except:    
                data["Category"] = "NaN"
            try:
                data_lat.append(response_crime[i]["location"]["latitude"])
                data["Latitude"] = data_lat
            except:
                data["Latitude"] = "NaN"
            try:
                data_lon.append(response_crime[i]["location"]["longitude"])
                data["Longitude"] = data_lon
            except:
                data["Longitude"] = "NaN"
            try:
                data_outcome.append(response_crime[i]["outcome_status"]["category"])
                data["Outcome"] = data_outcome
            except:
                data["Outcome"] = "NaN"
            try:
                data_month.append(response_crime[i]["month"])
                data["Date"] = data_month
            except:
                data["Date"] = "NaN"
            try:
                data["Neighbourhood"] = h
            except:
                data["Neighbourhood"] = "NaN"
                print(f"---At the end we fetched data for: {h} Neighbourhood---")

---Trying to get data for : BE001---
We've got 1755 coordinates to work with
Total coordinates that we work with is: 117
Date is: &date=2020-01
We've got 1755 coordinates to work with
Total coordinates that we work with is: 117
Date is: &date=2020-02
We've got 1755 coordinates to work with
Total coordinates that we work with is: 117
Date is: &date=2020-03
We've got 1755 coordinates to work with
Total coordinates that we work with is: 117
Date is: &date=2020-04
We've got 1755 coordinates to work with
Total coordinates that we work with is: 117
Date is: &date=2020-05
We've got 1755 coordinates to work with
Total coordinates that we work with is: 117
Date is: &date=2020-06
We've got 1755 coordinates to work with
Total coordinates that we work with is: 117
Date is: &date=2020-07
We've got 1755 coordinates to work with
Total coordinates that we work with is: 117
Date is: &date=2020-08
We've got 1755 coordinates to work with
Total coordinates that we work with is: 117
Date is: &date=2020-09


We've got 765 coordinates to work with
Total coordinates that we work with is: 77
Date is: &date=2020-04
We've got 765 coordinates to work with
Total coordinates that we work with is: 77
Date is: &date=2020-05
We've got 765 coordinates to work with
Total coordinates that we work with is: 77
Date is: &date=2020-06
We've got 765 coordinates to work with
Total coordinates that we work with is: 77
Date is: &date=2020-07
We've got 765 coordinates to work with
Total coordinates that we work with is: 77
Date is: &date=2020-08
We've got 765 coordinates to work with
Total coordinates that we work with is: 77
Date is: &date=2020-09
We've got 765 coordinates to work with
Total coordinates that we work with is: 77
Date is: &date=2020-10
We've got 765 coordinates to work with
Total coordinates that we work with is: 77
Date is: &date=2020-11
We've got 765 coordinates to work with
Total coordinates that we work with is: 77
Date is: &date=2020-12
---Trying to get data for : BE008---
We've got 1417 coo

We've got 597 coordinates to work with
Total coordinates that we work with is: 60
Date is: &date=2020-07
We've got 597 coordinates to work with
Total coordinates that we work with is: 60
Date is: &date=2020-08
We've got 597 coordinates to work with
Total coordinates that we work with is: 60
Date is: &date=2020-09
We've got 597 coordinates to work with
Total coordinates that we work with is: 60
Date is: &date=2020-10
We've got 597 coordinates to work with
Total coordinates that we work with is: 60
Date is: &date=2020-11
We've got 597 coordinates to work with
Total coordinates that we work with is: 60
Date is: &date=2020-12
---Trying to get data for : BE014---
We've got 538 coordinates to work with
Total coordinates that we work with is: 54
Date is: &date=2020-01
We've got 538 coordinates to work with
Total coordinates that we work with is: 54
Date is: &date=2020-02
We've got 538 coordinates to work with
Total coordinates that we work with is: 54
Date is: &date=2020-03
We've got 538 coor

We've got 2047 coordinates to work with
Total coordinates that we work with is: 137
Date is: &date=2020-11
We've got 2047 coordinates to work with
Total coordinates that we work with is: 137
Date is: &date=2020-12
---Trying to get data for : BE020---
We've got 1231 coordinates to work with
Total coordinates that we work with is: 124
Date is: &date=2020-01
We've got 1231 coordinates to work with
Total coordinates that we work with is: 124
Date is: &date=2020-02
We've got 1231 coordinates to work with
Total coordinates that we work with is: 124
Date is: &date=2020-03
We've got 1231 coordinates to work with
Total coordinates that we work with is: 124
Date is: &date=2020-04
We've got 1231 coordinates to work with
Total coordinates that we work with is: 124
Date is: &date=2020-05
We've got 1231 coordinates to work with
Total coordinates that we work with is: 124
Date is: &date=2020-06
We've got 1231 coordinates to work with
Total coordinates that we work with is: 124
Date is: &date=2020-07


We've got 752 coordinates to work with
Total coordinates that we work with is: 76
Date is: &date=2020-02
We've got 752 coordinates to work with
Total coordinates that we work with is: 76
Date is: &date=2020-03
We've got 752 coordinates to work with
Total coordinates that we work with is: 76
Date is: &date=2020-04
We've got 752 coordinates to work with
Total coordinates that we work with is: 76
Date is: &date=2020-05
We've got 752 coordinates to work with
Total coordinates that we work with is: 76
Date is: &date=2020-06
We've got 752 coordinates to work with
Total coordinates that we work with is: 76
Date is: &date=2020-07
We've got 752 coordinates to work with
Total coordinates that we work with is: 76
Date is: &date=2020-08
We've got 752 coordinates to work with
Total coordinates that we work with is: 76
Date is: &date=2020-09
We've got 752 coordinates to work with
Total coordinates that we work with is: 76
Date is: &date=2020-10
We've got 752 coordinates to work with
Total coordinate

...

We've got 813 coordinates to work with
Total coordinates that we work with is: 82
Date is: &date=2020-09
We've got 813 coordinates to work with
Total coordinates that we work with is: 82
Date is: &date=2020-10
We've got 813 coordinates to work with
Total coordinates that we work with is: 82
Date is: &date=2020-11
We've got 813 coordinates to work with
Total coordinates that we work with is: 82
Date is: &date=2020-12
---Trying to get data for : SH002---
We've got 826 coordinates to work with
Total coordinates that we work with is: 83
Date is: &date=2020-01
We've got 826 coordinates to work with
Total coordinates that we work with is: 83
Date is: &date=2020-02
We've got 826 coordinates to work with
Total coordinates that we work with is: 83
Date is: &date=2020-03
We've got 826 coordinates to work with
Total coordinates that we work with is: 83
Date is: &date=2020-04
We've got 826 coordinates to work with
Total coordinates that we work with is: 83
Date is: &date=2020-05
We've got 826 coor

---Trying to get data for : SH099---
We've got 508 coordinates to work with
Total coordinates that we work with is: 51
Date is: &date=2020-01
We've got 508 coordinates to work with
Total coordinates that we work with is: 51
Date is: &date=2020-02
We've got 508 coordinates to work with
Total coordinates that we work with is: 51
Date is: &date=2020-03
We've got 508 coordinates to work with
Total coordinates that we work with is: 51
Date is: &date=2020-04
We've got 508 coordinates to work with
Total coordinates that we work with is: 51
Date is: &date=2020-05
We've got 508 coordinates to work with
Total coordinates that we work with is: 51
Date is: &date=2020-06
We've got 508 coordinates to work with
Total coordinates that we work with is: 51
Date is: &date=2020-07
We've got 508 coordinates to work with
Total coordinates that we work with is: 51
Date is: &date=2020-08
We've got 508 coordinates to work with
Total coordinates that we work with is: 51
Date is: &date=2020-09
We've got 508 coor

We've got 664 coordinates to work with
Total coordinates that we work with is: 67
Date is: &date=2020-04
We've got 664 coordinates to work with
Total coordinates that we work with is: 67
Date is: &date=2020-05
We've got 664 coordinates to work with
Total coordinates that we work with is: 67
Date is: &date=2020-06
We've got 664 coordinates to work with
Total coordinates that we work with is: 67
Date is: &date=2020-07
We've got 664 coordinates to work with
Total coordinates that we work with is: 67
Date is: &date=2020-08
We've got 664 coordinates to work with
Total coordinates that we work with is: 67
Date is: &date=2020-09
We've got 664 coordinates to work with
Total coordinates that we work with is: 67
Date is: &date=2020-10
We've got 664 coordinates to work with
Total coordinates that we work with is: 67
Date is: &date=2020-11
We've got 664 coordinates to work with
Total coordinates that we work with is: 67
Date is: &date=2020-12
---Trying to get data for : WS001---
We've got 1708 coo

We've got 525 coordinates to work with
Total coordinates that we work with is: 53
Date is: &date=2020-08
We've got 525 coordinates to work with
Total coordinates that we work with is: 53
Date is: &date=2020-09
We've got 525 coordinates to work with
Total coordinates that we work with is: 53
Date is: &date=2020-10
We've got 525 coordinates to work with
Total coordinates that we work with is: 53
Date is: &date=2020-11
We've got 525 coordinates to work with
Total coordinates that we work with is: 53
Date is: &date=2020-12
---Trying to get data for : WS007---
We've got 1355 coordinates to work with
Total coordinates that we work with is: 136
Date is: &date=2020-01
We've got 1355 coordinates to work with
Total coordinates that we work with is: 136
Date is: &date=2020-02
We've got 1355 coordinates to work with
Total coordinates that we work with is: 136
Date is: &date=2020-03
We've got 1355 coordinates to work with
Total coordinates that we work with is: 136
Date is: &date=2020-04
We've got 

We've got 1775 coordinates to work with
Total coordinates that we work with is: 119
Date is: &date=2020-11
We've got 1775 coordinates to work with
Total coordinates that we work with is: 119
Date is: &date=2020-12
---Trying to get data for : WV004---
We've got 646 coordinates to work with
Total coordinates that we work with is: 65
Date is: &date=2020-01
We've got 646 coordinates to work with
Total coordinates that we work with is: 65
Date is: &date=2020-02
We've got 646 coordinates to work with
Total coordinates that we work with is: 65
Date is: &date=2020-03
We've got 646 coordinates to work with
Total coordinates that we work with is: 65
Date is: &date=2020-04
We've got 646 coordinates to work with
Total coordinates that we work with is: 65
Date is: &date=2020-05
We've got 646 coordinates to work with
Total coordinates that we work with is: 65
Date is: &date=2020-06
We've got 646 coordinates to work with
Total coordinates that we work with is: 65
Date is: &date=2020-07
We've got 646 

In [74]:
data

{'Category': ['anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'anti-social-behaviour',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'burglary',
  'criminal-damage-arson',
  'criminal-damage-arson',
  'criminal-damage-arson',
  'criminal-damage-arson',
  'criminal-damage-arson',
  'criminal-damage-arson',
  'criminal-damage-arson',
  'crimi

In [72]:
#Here we change our dictionary into Series, i works on headers, j works on the rest of data rows. 
#Check out documentation for .items function
data_df = pd.DataFrame({i:pd.Series(j) for i, j in data.items()})


,Category,Latitude,Longitude,Outcome,Date,Neighbourhood


In [81]:
data_df

,Category,Latitude,Longitude,Outcome,Date,Neighbourhood
0,anti-social-behaviour,52.522695,-1.842639,Investigation complete; no suspect identified,2020-01,WV008
1,anti-social-behaviour,52.532120,-1.826257,Investigation complete; no suspect identified,2020-01,NaN
2,anti-social-behaviour,52.529918,-1.826177,Investigation complete; no suspect identified,2020-01,NaN
3,anti-social-behaviour,52.530821,-1.849008,Investigation complete; no suspect identified,2020-01,NaN
4,anti-social-behaviour,52.522863,-1.840708,Investigation complete; no suspect identified,2020-01,NaN
...,...,...,...,...,...,...
322754,violent-crime,52.615633,-2.055535,NaN,2020-12,NaN
322755,violent-crime,52.610128,-2.061095,NaN,2020-12,NaN
322756,other-crime,52.612523,-2.054423,NaN,2020-12,NaN
322757,other-crime,52.600143,-2.056977,NaN,2020-12,NaN


In [86]:
#I dont know why but it returns noting here, lemme know if you find a way how to get it to show NaN columns
data_df.loc[data_df["Outcome"]=="NaN"]

,Category,Latitude,Longitude,Outcome,Date,Neighbourhood


In [ ]:
#Time to clean data and do fancy statistics and such. Good luck we can do it, one last push. 